![20220715132147](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220715132147.png)

![20220715132250](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220715132250.png)

## K-means

通过把样本分离成 n 个`具有相同方差的类`的方式来聚集数据，`基于距离`的模型。

算法目标--确定K，并最小化簇内平方和

![20220715133333](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220715133333.png)

1. 选取K个质心；
2. 将每个样本分配到其最近的质心；
3. 通过分配给每个质心的所有样本的平均值来创建新的质心。计算旧的和新的质心之间的差异，直到该值小于阈值（质心不再显著移动）

所以，`K-means对于圆形区域的聚类效果更好`，因为是基于均值不断更新质心，如果不是圆形区域，则可考虑基于分布的高斯混合进行聚类。

In [2]:
from sklearn.cluster import KMeans
import numpy as np

X = np.array([[1, 2], [1, 4], [1, 0],[10, 2], [10, 4], [10, 0]])

# 默认 k-means++ 
kmeans = KMeans(n_clusters=2, random_state=0,init='k-means++').fit(X)
# 每个数据点所属类
print(kmeans.labels_) # 或者直接 kmeans.fit_predict(X)
print(kmeans.predict([[0, 0], [12, 3]]))
# 聚类中心
print(kmeans.cluster_centers_)

[1 1 1 0 0 0]
[1 0]
[[10.  2.]
 [ 1.  2.]]


## Mini Batch K-Means